# Batch Process All Segments - Full Day, Max Depth

This notebook orchestrates the full pipeline for all market segments:
- Processes full trading day (not just 10-minute windows)
- Detects and uses maximum available depth (L5, L10, L20, etc.)
- Saves all intermediate products to Google Drive
- Raw data stays in Colab local SSD

**Prerequisites:** Run notebook `0_env_db_prep.ipynb` first to extract raw data to Colab local.

In [19]:
# Check if Drive is mounted
import os
from pathlib import Path

drive_mount = Path('/content/drive')
if not drive_mount.exists():
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted")
else:
    print("Google Drive already mounted")

Google Drive already mounted


In [20]:
# Set paths
REPO_DIR = "/content/drive/MyDrive/00_EUREX/eurex-liquidity-demo"
RAW_LOCAL = "/content/Sample_Eurex_20201201_10MktSegID"  # Colab local SSD
OUT_DRIVE = f"{REPO_DIR}/data_segments"  # Output to Drive
DATE = "20201201"

print(f"Repo: {REPO_DIR}")
print(f"Raw data (local): {RAW_LOCAL}")
print(f"Output (Drive): {OUT_DRIVE}")

Repo: /content/drive/MyDrive/00_EUREX/eurex-liquidity-demo
Raw data (local): /content/Sample_Eurex_20201201_10MktSegID
Output (Drive): /content/drive/MyDrive/00_EUREX/eurex-liquidity-demo/data_segments


## Option 1: Process specific segments

Choose segments based on your interview focus:
- **48**: Small, quick test (1.22 MB DI)
- **821**: Medium activity (54.77 MB)
- **589, 688, 1373**: High activity (200-400 MB)
- **1176**: Very high activity, likely major index (9.6 GB!)

## Process Segments with Auto-Depth Detection

The script automatically uses optimal depth levels for each segment from `segment_depth_summary.json`.

**Options:**
- `AUTO_DEPTH=True`: Use per-segment depth from analysis (RECOMMENDED)
- `AUTO_DEPTH=False, MAX_LEVELS=6`: Use fixed depth for all segments

In [21]:
# OPTIMIZED MODE: Process segments, excluding large ones that cause OOM

# Segment detection
AUTO_DETECT = False  # ⚠️ Manual segment list (excluding large ones)
SEGMENTS = "688"  # Segments that fit in memory

# Depth detection
AUTO_DEPTH = True    # ✅ Use optimal depth per segment from depth summary
MAX_LEVELS = None    # Not used when AUTO_DEPTH=True

# Processing options
PARALLEL = 2         # Process 2 segments simultaneously (recommended)
SKIP_EXISTING = True # ⚡ Skip segments that are already completed

print("🚀 OPTIMIZED MODE - SKIP LARGE SEGMENTS")
print("="*70)
print("Processing segments: " + SEGMENTS)
print("Skipping: 688, 1373, 1176 (too large for runtime)")
print(f"Parallelism: {PARALLEL} segments at once")
print(f"Skip existing: {SKIP_EXISTING} (safe to rerun)")
print(f"Estimated time: 1-2 hours")
print(f"Expected output: ~500 MB in Drive")
print("="*70)
print()

# Build command
cmd_parts = [
    f'python "{REPO_DIR}/scripts/process_all_segments.py"',
]

if AUTO_DETECT:
    cmd_parts.append("--auto-detect")
else:
    cmd_parts.append(f"--segments {SEGMENTS}")

cmd_parts.extend([
    f'--src-local "{RAW_LOCAL}"',
    f'--out-drive "{OUT_DRIVE}"',
    f'--date {DATE}',
])

if AUTO_DEPTH:
    cmd_parts.append("--auto-depth")
elif MAX_LEVELS is not None:
    cmd_parts.append(f"--max-levels {MAX_LEVELS}")

cmd_parts.append(f"--parallel {PARALLEL}")

if SKIP_EXISTING:
    cmd_parts.append("--skip-existing")

# Execute
cmd = " \\\n  ".join(cmd_parts)
print("Command:")
print(cmd)
print("\n" + "="*70 + "\n")

!{cmd}

🚀 OPTIMIZED MODE - SKIP LARGE SEGMENTS
Processing segments: 688
Skipping: 688, 1373, 1176 (too large for runtime)
Parallelism: 2 segments at once
Skip existing: True (safe to rerun)
Estimated time: 1-2 hours
Expected output: ~500 MB in Drive

Command:
python "/content/drive/MyDrive/00_EUREX/eurex-liquidity-demo/scripts/process_all_segments.py" \
  --segments 688 \
  --src-local "/content/Sample_Eurex_20201201_10MktSegID" \
  --out-drive "/content/drive/MyDrive/00_EUREX/eurex-liquidity-demo/data_segments" \
  --date 20201201 \
  --auto-depth \
  --parallel 2 \
  --skip-existing


[INFO] Loading depth summary for auto-detection...
[INFO] Loaded depth info for 10 segments
[INFO] Checking for existing completed segments...
[INFO] 1 segments remaining to process

BATCH PROCESSING 1 SEGMENTS
Date: 20201201
Max levels: AUTO (per-segment from depth summary)
  Segment   688: L3
Parallelism: 2
Segments: [688]

[INFO] Using 2 parallel workers

Starting segment 688

Processing Segment 688 - Full D

## 🔧 Fix Incomplete Segments

**Problem:** Some segments completed parsing (have snapshots) but failed during aggregation.

**Common Issues:**
- **Segment 1209**: NoneType error (FIXED in aggregate_l5.py)
- **Segment 1374**: Security ID parsing error
- **Segments 50, 1373**: OOM or interrupted

**Next Steps:**
1. Run **Cell 7** to auto-fix segments with snapshots but no aggregates
2. Run **Cell 8** to manually retry completely failed segments (one at a time)

In [40]:
# Fix incomplete segments manually
# Run this if some segments completed snapshots but not aggregation

import os
from pathlib import Path

# Check which segments need aggregation fix
segments_to_fix = []

for seg_dir in Path(OUT_DRIVE).glob("seg_*"):
    seg_id = seg_dir.name.split("_")[1]
    
    # Check if has snapshots but no aggregates
    has_snapshots = False
    has_aggregates = False
    
    for lx_dir in seg_dir.glob("l*"):
        if list(lx_dir.glob("*_snapshots_*.parquet")):
            has_snapshots = True
        if list(lx_dir.glob("*_agg_1s_*.parquet")):
            has_aggregates = True
    
    if has_snapshots and not has_aggregates:
        segments_to_fix.append(seg_id)

if segments_to_fix:
    print(f"🔧 Found {len(segments_to_fix)} segments needing aggregation fix:")
    print(f"   Segments: {', '.join(segments_to_fix)}")
    print()
    
    for seg_id in segments_to_fix:
        seg_dir = Path(OUT_DRIVE) / f"seg_{seg_id}"
        
        # Find the snapshot file
        snapshot_file = None
        di_file = None
        mapping_file = None
        
        for lx_dir in seg_dir.glob("l*"):
            snapshot_files = list(lx_dir.glob("*_snapshots_*.parquet"))
            if snapshot_files:
                snapshot_file = snapshot_files[0]
                break
        
        di_files = list(seg_dir.glob("DI_*.csv"))
        if di_files:
            di_file = di_files[0]
        
        mapping_files = list(seg_dir.glob("di_mapping.json"))
        if mapping_files:
            mapping_file = mapping_files[0]
        
        if snapshot_file and di_file and mapping_file:
            print(f"▶ Fixing segment {seg_id}...")
            cmd = f'python "{REPO_DIR}/scripts/aggregate_l5.py" \\\n'
            cmd += f'  --seg {seg_id} \\\n'
            cmd += f'  --l5 "{snapshot_file}" \\\n'
            cmd += f'  --di "{di_file}" \\\n'
            cmd += f'  --mapping "{mapping_file}" \\\n'
            cmd += f'  --out "{seg_dir}"'
            
            print(cmd)
            print()
            os.system(cmd)
            print()
        else:
            print(f"⚠️  Segment {seg_id}: Missing required files")
            print(f"   Snapshot: {snapshot_file}")
            print(f"   DI: {di_file}")
            print(f"   Mapping: {mapping_file}")
    
    print("✅ Aggregation fixes complete!")
else:
    print("✅ All segments already have aggregates!")

✅ All segments already have aggregates!


## Resume Segment 50 from Checkpoint

The parsing and batching completed successfully, but the final merge OOM'd. Use this cell to resume from the 21 merged batch files.

In [17]:
# Resume segment 50 merge from existing batch files
# This uses an ultra-conservative approach: merges 5 files at a time

SEG = 50
TEMP_DIR = f"{OUT_DRIVE}/seg_{SEG}/l5/_temp_chunks"
OUT_DIR = f"{OUT_DRIVE}/seg_{SEG}"

print(f"🔄 Resuming segment {SEG} merge from checkpoint")
print("="*70)
print(f"Temp dir: {TEMP_DIR}")
print(f"Output dir: {OUT_DIR}")
print("Strategy: Merge 5 batches at a time to minimize memory")
print("="*70)
print()

!python "{REPO_DIR}/scripts/resume_merge_seg50.py" \
  --seg {SEG} \
  --temp-dir "{TEMP_DIR}" \
  --out-dir "{OUT_DIR}" \
  --levels 5

🔄 Resuming segment 50 merge from checkpoint
Temp dir: /content/drive/MyDrive/00_EUREX/eurex-liquidity-demo/data_segments/seg_50/l5/_temp_chunks
Output dir: /content/drive/MyDrive/00_EUREX/eurex-liquidity-demo/data_segments/seg_50
Strategy: Merge 5 batches at a time to minimize memory

[INFO] Found 21 merged batch files
[INFO] Starting ultra-conservative merge (5 files at a time)...
  Super-batch 1/5: merging 5 files...
    Loading merged_0000.parquet...
    Loading merged_0020.parquet...
    Loading merged_0040.parquet...
    Loading merged_0060.parquet...
    Loading merged_0080.parquet...
    Concatenating 5 dataframes...
    Sorting 5,000,000 rows...
    Writing to final_batch_01.parquet...
    ✅ Super-batch 1 complete, freed memory
  Super-batch 2/5: merging 5 files...
    Loading merged_0100.parquet...
    Loading merged_0120.parquet...
    Loading merged_0140.parquet...
    Loading merged_0160.parquet...
    Loading merged_0180.parquet...
    Concatenating 5 dataframes...
/conten

## Check processing results

Review the batch summary and individual segment outputs.

In [20]:
import json
import pandas as pd

# Load batch summary
summary_path = f"{OUT_DRIVE}/batch_summary.json"
if os.path.exists(summary_path):
    with open(summary_path) as f:
        summary = json.load(f)
    
    print(f"Processed {len(summary['results'])} segments in {summary['overall_duration_sec']}s")
    print(f"Success: {summary['success_count']}, Failed: {summary['fail_count']}")
    
    # Show results table
    df = pd.DataFrame(summary['results'])
    display(df)
else:
    print("No summary found yet. Run processing first.")

Processed 5 segments in 320s
Success: 1, Failed: 4


,segment,success,duration_sec,exit_code
0,688,False,11,1
1,1209,True,7,0
2,50,False,228,1
3,1374,False,6,1
4,1373,False,302,1


## Inspect a specific segment's output

Check the files generated for one segment.

In [21]:
# Pick a segment to inspect
SEG = 821
seg_dir = f"{OUT_DRIVE}/seg_{SEG}"

if os.path.exists(seg_dir):
    print(f"Output for segment {SEG}:")
    !ls -lh "{seg_dir}"
    
    # Show metadata
    metadata_path = f"{seg_dir}/metadata.json"
    if os.path.exists(metadata_path):
        with open(metadata_path) as f:
            meta = json.load(f)
        print("\nMetadata:")
        for k, v in meta.items():
            print(f"  {k}: {v}")
    
    # Check L{depth} subdirectory
    l_dirs = !ls -d "{seg_dir}"/l*
    if l_dirs:
        print(f"\nDepth subdirectories:")
        for ld in l_dirs:
            print(f"\n{ld}:")
            !ls -lh "{ld}"
else:
    print(f"Segment {SEG} not yet processed")


Depth subdirectories:

/content/drive/MyDrive/00_EUREX/eurex-liquidity-demo/data_segments/seg_821/l5:
total 59M
-rw------- 1 root root  28M Nov 16 15:00 l5_agg_1s_seg821.csv
-rw------- 1 root root 4.8M Nov 16 14:59 l5_agg_1s_seg821.parquet
-rw------- 1 root root  22M Nov 16 14:57 l5_snapshots_seg821.csv
-rw------- 1 root root 4.3M Nov 16 14:57 l5_snapshots_seg821.parquet


## Quick preview of aggregated data

Load and inspect the 1-second aggregates for a segment.

In [22]:
SEG = 589

# Find the aggregates file (L{depth} may vary)
import glob
agg_files = glob.glob(f"{OUT_DRIVE}/seg_{SEG}/l*/l*_agg_1s_seg{SEG}.parquet")

if agg_files:
    df = pd.read_parquet(agg_files[0])
    print(f"Loaded {len(df)} rows from {agg_files[0]}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nFirst few rows:")
    display(df.head(10))
    
    print(f"\nBasic stats:")
    display(df[['spread_abs', 'spread_rel', 'imbalance_l1', 'total_volume']].describe())
else:
    print(f"No aggregates found for segment {SEG}")

Loaded 153058 rows from /content/drive/MyDrive/00_EUREX/eurex-liquidity-demo/data_segments/seg_589/l5/l5_agg_1s_seg589.parquet

Columns: ['security_id', 'ts_s', 'best_bid', 'best_ask', 'bid_size_1', 'ask_size_1', 'spread_abs', 'spread_rel', 'midprice', 'imbalance_l1', 'microprice_l1', 'total_bid_volume', 'total_ask_volume', 'avg_bid_price', 'avg_ask_price', 'imbalance_l5', 'microprice_l5', 'depth_ratio', 'total_volume', 'volume_ratio_l1_to_l5', 'update_count', 'cancel_count']

First few rows:


,security_id,ts_s,best_bid,best_ask,bid_size_1,ask_size_1,spread_abs,spread_rel,midprice,imbalance_l1,...,total_ask_volume,avg_bid_price,avg_ask_price,imbalance_l5,microprice_l5,depth_ratio,total_volume,volume_ratio_l1_to_l5,update_count,cancel_count
0,5035771,1606781706,13406.0,13407.5,1.0,1.0,1.5,0.000112,13406.75,0.000000,...,10.0,13380.650000,13403.950000,0.000000,13392.300000,1.000000,20.0,0.100000,517,0
1,5035771,1606781707,13404.5,13353.5,1.0,1.0,-51.0,-0.003812,13379.00,0.000000,...,9.0,13397.800000,13394.444444,0.052632,13396.033918,0.900000,19.0,0.105263,231,0
2,5035771,1606781708,13391.5,13399.5,7.0,1.0,8.0,0.000597,13395.50,0.750000,...,17.0,13392.961538,13398.147059,0.209302,13396.096969,0.653846,43.0,0.186047,134,0
3,5035771,1606781709,13337.0,13406.0,1.0,1.0,69.0,0.005160,13371.50,0.000000,...,9.0,13396.205882,13400.888889,0.307692,13399.267848,0.529412,26.0,0.076923,95,0
4,5035771,1606781710,13428.0,13436.0,1.0,9.0,8.0,0.000596,13432.00,-0.800000,...,20.0,13397.428571,13427.450000,-0.481481,13405.211905,2.857143,27.0,0.370370,213,0
5,5035771,1606781711,13427.0,13435.0,1.0,7.0,8.0,0.000596,13431.00,-0.750000,...,14.0,13393.545455,13422.428571,-0.120000,13406.254026,1.272727,25.0,0.320000,41,0
6,5035771,1606781712,13414.5,13367.0,1.0,5.0,-47.5,-0.003547,13390.75,-0.666667,...,17.0,13403.181818,13393.264706,-0.214286,13399.285810,1.545455,28.0,0.214286,32,0
7,5035771,1606781713,13333.5,13421.5,1.0,3.0,88.0,0.006578,13377.50,-0.500000,...,8.0,13388.000000,13405.812500,-0.066667,13396.312500,1.142857,15.0,0.266667,7,0
8,5035771,1606781714,13409.0,13417.0,1.0,8.0,8.0,0.000596,13413.00,-0.777778,...,26.0,13399.250000,13410.211538,-0.368421,13402.711538,2.166667,38.0,0.236842,107,0
9,5035771,1606781715,13337.0,13397.5,1.0,4.0,60.5,0.004526,13367.25,-0.600000,...,15.0,13372.666667,13393.800000,-0.111111,13382.059259,1.250000,27.0,0.185185,172,0



Basic stats:


,spread_abs,spread_rel,imbalance_l1,total_volume
count,153054.000000,152497.000000,153054.000000,153058.000000
mean,5.691550,0.001197,0.012406,18.369938
std,13.336317,0.079737,0.431020,13.505417
min,-75.000000,-7.714286,-0.967742,1.000000
25%,1.500000,0.000112,-0.333333,7.000000
50%,3.500000,0.000261,0.000000,18.000000
75%,8.500000,0.000634,0.333333,27.000000
max,421.000000,5.619048,0.964912,488.000000


## Next Steps

1. Go to `3_db_setup.ipynb` and check the DuckDB setups
2. Create Power BI dashboard with interactive filters:
   - Segment/Product selector
   - Granularity toggle (1s, 5s, 1m)
   - Depth level selector (L1, L5, L10, L20)
   - Visual type selector (spread, imbalance, volume, etc.)
3. Export dashboard as PBIP for version control